In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
#!python
from numpy import *
from numpy.random import *
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import sys
import cPickle as pickle
from   random import *
import scipy.stats
import matplotlib.pyplot as plt
import pyparticleest.utils.kalman as kalman
from IPython.display import Image

import numpy as np
import scipy as sp
import scipy.special
import scipy.linalg

import os
os.getcwd()
import sys

sys.path.insert(1,'/Users/zyzrdfz/Desktop/gwungwun/gsl-2.1/CythonGSL')
from random1 import resample

# sys.path.insert(1,'/Users/zyzrdfz/Documents/ss_model_mixture/Cython_Code')
# from fastloop import resample

In [26]:
mat = np.loadtxt('train_data.txt')
Y = mat[:,1:]
T = Y.shape[1]
n = Y.shape[0]

In [3]:
df = pd.read_csv('thaldata.csv',header=None)
Y = df.values.reshape(3000)
# Y = Y[500:800]
T = len(Y)

In [4]:
print Y.shape

(3000,)


In [5]:
def measure(X, Y, trial):
    coefficient = scipy.special.binom(trial,Y)
    # log-pdf of p(y_t|x_t)
    return np.log(coefficient) + X * Y - trial * np.log(1+np.exp(X))

In [8]:
def gaussian(X, Q):
    #log-pdf of p(x_t|x_{t-1})
    return -0.5 * (l2pi + np.log(Q) + (X ** 2) / Q)

In [23]:
def filtering2(T, N, Q, Y, trial):
    samples = np.random.normal(0,1,[T,N])
    X = np.empty([T,N])
    ancestor = np.empty([T,N])
    w = np.empty([T,N])
    z = np.empty([T,N])
    pi = np.empty([T,N])
    W = np.empty([T,N])
    ##Initialization
    t = 0
    x_prior = 0.0
    s2_prior = Q
    x_post = x_prior + s2_prior*(Y[t] - trial * np.exp(x_prior)/(1.0 + np.exp(x_prior)))
    s2_post = 1.0/(1.0/s2_prior + trial * np.exp(x_post)/(1 + np.exp(x_post))**2)
    X[t,:] = x_post + np.sqrt(s2_post + 0* Q) * samples[t,:]
    w[t,:] = measure(Y[t], X[t,:], trial) + gaussian(X[t,:], Q) - gaussian(X[t,:] - x_post, s2_post + 0*Q)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    z[t,:] = W[t,:] * np.exp(gaussian(Y[t+1]-X[t,:], s2_post + Q))
    pi[t,:] = z[t,:]/np.sum(z[t,:])
    ancestor[t,:]= X[t,resample(pi[t,:])]
    
    for tt in range(T-2):
        t=tt+1
        x_prior = ancestor[tt,:]
        s2_prior = Q
        x_post = x_prior + s2_prior*(Y[t] - trial * np.exp(x_prior)/(1.0 + np.exp(x_prior)))
        s2_post = 1.0/(1.0/s2_prior + trial * np.exp(x_post)/(1 + np.exp(x_post))**2)
        X[t,:] = x_post + np.sqrt(s2_post + 0*Q) * samples[t,:]
        w[t,:] = measure(Y[t], X[t,:], trial) + gaussian(X[t,:] - ancestor[tt,:], Q) - gaussian(X[t,:] - x_post, s2_post + 0*Q) - gaussian(Y[t]- ancestor[tt,:],s2_post + Q)    
        W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
        W[t,:] /= np.sum(W[t,:])
        z[t,:] = W[t,:] * np.exp(gaussian(Y[t+1]- X[t,:],s2_post + Q))
        pi[t,:] = z[t,:]/np.sum(z[t,:])
        ancestor[t,:]= X[t,resample(pi[t,:])]
    
    t=T-1
    x_prior = ancestor[t-1,:]
    s2_prior = Q
    x_post = x_prior + s2_prior*(Y[t] - trial * np.exp(x_prior)/(1.0 + np.exp(x_prior)))
    s2_post = 1.0/(1.0/s2_prior + trial * np.exp(x_post)/(1 + np.exp(x_post))**2)
    X[t,:] = x_post + np.sqrt(s2_post + 0*Q) * samples[t,:]
    w[t,:] = measure(Y[t], X[t,:], trial) + gaussian(X[t,:] - ancestor[t-1,:], Q) - gaussian(X[t,:] - x_post, s2_post + 0*Q)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    z[t,:] = W[t,:]
    ancestor[t,:]= X[t,resample(W[t,:])]
    
    return X, w, W, ancestor, z

In [24]:
sigma2e = np.arange(0.1, 0.51, 0.5)
for i in range(len(sigma2e)):
    Q = sigma2e[i]
    [X, w, W, ancestor, z] = filtering2(T, N, Q, Y, trial)   
#     maxlogweight = np.max(w, axis=1)
#     weight = np.exp(w - np.reshape(np.repeat(maxlogweight, N),(T, N)))
#     loglikelihood_withresampling[i] = np.sum(np.log(np.sum(weight, axis = 1)/N))+np.sum(maxlogweight)
    
    loglikelihood_withresampling = np.sum(np.log(np.sum(z[:T-2], axis = 1))) + np.sum(np.log(np.sum(np.exp(w[:T-2]), axis =1)/N))

In [25]:
print loglikelihood_withresampling

nan


In [57]:
prior=[0.5, 0.5]
sigma = [0.2, 0.3]

In [7]:
# Forward particles
N = 1000
# Backward trajectories
M = 30
# trials
trial = 50
# cluster
cluster = 2
#iterations
iteration = 10
#
l2pi = math.log(2 * math.pi)
#
sigma_ls = np.empty([iteration,cluster])
prior_ls = np.empty([iteration,cluster])

In [43]:
def measure(X, Y, trial):
    coefficient = scipy.special.binom(trial,Y)
    # log-pdf of p(y_t|x_t)
    return np.log(coefficient) + X * Y - trial * np.log(1+np.exp(X))

In [44]:
def gaussian(X, Q):
    #log-pdf of p(x_t|x_{t-1})
    return -0.5 * (l2pi + np.log(Q) + (X ** 2) / Q)

In [45]:
def Do_Kalman_Likelihood_Bernoulli_LaplaceMAP(dN, sigma2e, tol=1e-8, trials=1.):
    """MAP solution, inverse covariance matrix, and marginal loglikelihood of state-space model
    computed using Laplace approximation around MAP state.

    :param dN: Observations (K,)
    :param sigma2e: Variance of process noise
    :param tol: Convergence criterion on the gradient of the log-likelihood
    :param trials: Number of trials for binomial observations (1 for Bernoulli)
    :return: x_map, U, marginal_loglikelihood, joint_loglikelihood
    """
    x = np.zeros(dN.shape)
    dN = dN.astype(float)
    while True:
        # Build gradient of joint
        d2x = np.convolve(x, [-1, 2, -1])[1:-1]
        d2x[-1] -= x[-1]
        G = -dN + trials * (1. / (1. + np.exp(-x))) + d2x / sigma2e
        # Build Hessian of joint
        D = trials / (np.exp(x) + 2. + np.exp(-x)) + 2. / sigma2e
        D[-1] -= 1. / sigma2e
        B = -np.ones(len(D)) / sigma2e
        B[0] = 0.
        U = sp.linalg.cholesky_banded((B, D), lower=False)
        # Check convergence
        if np.dot(G, G) < tol:
            x_map = x
            break
        # Update estimate of map
        x -= sp.linalg.cho_solve_banded([U, False], G)

    # Compute joint and marginal probabilities
    joint_loglikelihood = (np.sum(np.log(sp.special.binom(trials, dN)) + dN * x_map - trials * np.log(1 + np.exp(x_map))) -
                           .5 * ((np.sum(np.diff(x_map)**2) + x_map[0]**2) / sigma2e + len(dN) * np.log(2*np.pi*sigma2e)))
    marginal_loglikelihood = len(dN)/2. * np.log(2*np.pi) + joint_loglikelihood - np.sum(np.log(U[-1]))
    return x_map, U, marginal_loglikelihood, joint_loglikelihood

In [60]:
def smoothing(T, N, Q, M, X, W):
    w_back = np.empty(N)
    W_BACK = np.empty([T,N])
    smoother = np.empty([T,M])
    multinomialsamples = np.random.multinomial(M, ones(N)/N)
    X_nonzero = X[T-1, multinomialsamples>0]
    smoother[T-1,:] = np.repeat(X_nonzero, multinomialsamples[multinomialsamples>0])


    for tt in range(T-1):
        t = T-2 - tt
        for j in range(M):
            w_back = ones(N)/N * np.exp(gaussian(smoother[t+1,j]-X[t+1,:], Q))
            W_BACK[t,:] = w_back/np.sum(w_back)
            multinomialsamples = np.random.multinomial(1, W_BACK[t,:])
            smoother[t,j] = X[t, multinomialsamples>0]


    return smoother

In [61]:
def filtering2(T, N, Q, Y, mode, variance):
    samples = np.random.normal(0,1,[T,N])
    X = np.reshape(np.repeat(mode,N),[T,N]) + np.reshape(np.repeat(np.sqrt(variance),N),[T,N]) * samples
    ancestor = np.empty([T,N])
    w = np.empty([T,N])
    z = np.empty([T,N])
    pi = np.empty([T,N])
    W = np.empty([T,N])
    ##Initialization
    t = 0
    w[t,:] = measure(X[t,:], Y[t], trial) + gaussian(X[t,:], Q) - gaussian(X[t,:] - mode[t], variance[t])
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    z[t,:] = W[t,:] * np.exp(measure(ancestor[t,:], Y[t+1], trial) )
    pi[t,:] = z[t,:]/np.sum(z[t,:])
    ancestor[t,:]= X[t,resample(pi[t,:])]
    
    for tt in range(T-2):
        t=tt+1
        w[t,:] = measure(X[t,:], Y[t], trial) + gaussian(X[t,:] - ancestor[tt,:], Q) - gaussian(X[t,:] - mode[t], variance[t]) - measure(ancestor[tt,:], Y[t], trial)    
        W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
        W[t,:] /= np.sum(W[t,:])
        z[t,:] = W[t,:] * np.exp(measure(ancestor[t,:], Y[t+1], trial) )
        pi[t,:] = z[t,:]/np.sum(z[t,:])
        ancestor[t,:]= X[t,resample(pi[t,:])]
    
    t=T-1
    w[t,:] = measure(X[t,:], Y[t], trial) + gaussian(X[t,:] - ancestor[t-1,:], Q) - gaussian(X[t,:] - mode[t], variance[t])
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    z[t,:] = W[t,:]
    pi[t,:] = z[t,:]/np.sum(z[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]
    
    return X, w, W, ancestor, z, pi

In [62]:
#%%time
sigmatmp = np.zeros([cluster,n])
loglikelihood = np.empty([cluster,n])
posterior_indicator = np.empty([cluster,n])
for i in range(iteration):
    for j in range(n):
        for c in range(cluster):
            Q = sigma[c]
            [x_map, U, marginal_loglikelihood, joint_loglikelihood] = Do_Kalman_Likelihood_Bernoulli_LaplaceMAP(Y[j,:], Q, 1e-8, trial) 
            mode = x_map
            variance = U[1]
            [X, w, W, ancestor, z, pi] = filtering2(T, N, Q, Y[j,:], mode, variance) 
            est_smooth = smoothing(T, N, Q, M, ancestor, pi)
            loglikelihood[c,j] = np.sum(np.log(np.sum(z[:T-2], axis = 1))) + np.sum(np.log(np.sum(np.exp(w[:T-2]), axis =1)/N))
            posteriorcovariance = np.array(np.diff(est_smooth,axis=0))**2
            sigmatmp[c,j] = np.sum(posteriorcovariance/(M*T))

    maxloglikelihood = np.max(loglikelihood, axis=0)
    www = np.exp(loglikelihood - maxloglikelihood)
    likelihood= np.sum(np.log(np.sum(www * np.reshape(np.repeat(prior,n),(cluster,n)), axis=0))) + np.sum(maxloglikelihood)
    print likelihood
    posterior_indicator = np.exp(loglikelihood - np.max(loglikelihood, axis=0)) * np.reshape(np.repeat(prior,n),(cluster,n))
    posterior_indicator = posterior_indicator/np.sum(posterior_indicator, axis=0)
    prior = np.sum(posterior_indicator,axis=1)/np.sum(posterior_indicator)
    sigma = np.sum(sigmatmp * posterior_indicator, axis=1)/np.sum(posterior_indicator,axis=1)
    sigma_ls[i,:]=sigma
    prior_ls[i,:]=prior

-85610.4192675
-101115.064767
-110435.997224
-111015.006138
-111477.331451
-111588.166946
-111048.758876


ValueError: array must not contain infs or NaNs

In [63]:
prior_ls

array([[  7.50035572e-001,   2.49964428e-001],
       [  1.00000000e+000,   4.42663643e-028],
       [  1.00000000e+000,   2.95978695e-146],
       [  1.00000000e+000,   2.71201978e-211],
       [  1.00000000e+000,   1.00826089e-231],
       [  1.00000000e+000,   3.79761198e-281],
       [  1.00000000e+000,   0.00000000e+000],
       [  1.00000000e+000,   1.00000000e+000],
       [  7.14864341e-077,   1.00000000e+000],
       [  1.00000000e+000,   1.00000000e+000]])